In [20]:
import nuclio 
import mlrun
import os

In [21]:
import storey

In [3]:
%nuclio env -c V3IO_ACCESS_KEY=${V3IO_ACCESS_KEY}
%nuclio env -c V3IO_USERNAME=${V3IO_USERNAME}
%nuclio env -c V3IO_API=${V3IO_API}

In [4]:
%%nuclio cmd -c
pip install beautifulsoup4
pip install pandas
pip install v3io_frames
pip install storey==0.4.0

In [5]:
%%nuclio config 
kind = "nuclio"
spec.build.baseImage = "mlrun/mlrun"

%nuclio: setting kind to 'nuclio'
%nuclio: setting spec.build.baseImage to 'mlrun/ml-models'


In [6]:
%nuclio cmd -c pip install v3io

In [7]:
# nuclio: start-code

In [22]:
import mlrun.feature_store as fs
from mlrun.feature_store.steps import *
import mlrun
import pandas as pd
import numpy
import os
import math

In [23]:
def rm_inf(data):
    print(data)
    data = [0 if item == math.inf or item == -math.inf else item for item in data ]
    #data = [0 for item in data if item == -math.inf]
    return data

In [24]:
def handler(context, event):
    context.logger.info("Reading features from feature vector")
    
    # Reading the data from feature service
    df = fs.get_offline_features("stocks-vec").to_dataframe()
    df = df.dropna()
    
    data = df.values
    
    columns = [{'text': key, 'type': 'object'} for key in df.columns]
    
    response = [{'columns': columns,
                'rows': data,
                'type': 'table'}]
    return response

In [25]:
def init_context(context):
    # Setting our enviornment
    context.logger.info("Initalizing context & constructing featrure vector")
    setattr(context, 'PROJECT_NAME', os.getenv('PROJECT_NAME', 'stocks2-' + os.getenv('V3IO_USERNAME')))
    mlrun.set_environment(project=context.PROJECT_NAME)
    
    sym_to_url = {'GOOGL': 'google-inc', 'MSFT': 'microsoft-corp', 'AMZN': 'amazon-com-inc',
                  'AAPL': 'apple-computer-inc', 'INTC' : 'intel-corp'}
    setattr(context, 'sym_to_url', sym_to_url)
    setattr(context, 'stocks_vec', "stocks-vec")
    
    # Declaring the features we would like to add to our vector
    features = ["stocks.Opens_min_1h",
                "stocks.Opens_max_1h",
                "stocks.Volumes_min_1h",
                "stocks.Volumes_max_1h",
                "stocks.Open",
                "stocks.High",
                "stocks.Low",
                "stocks.Close",
                "stocks.Volume",
                "stocks.ticker",
                "news.Sentiment"]
    
    # Making our vector
    vector = fs.FeatureVector(context.stocks_vec, features, description="stocks demo feature vector")
    vector.save()

In [26]:
# nuclio: end-code

In [27]:
mlrun.set_environment(project="stocks3-dani")

('stocks3-dani', 'v3io:///projects/{{run.project}}/artifacts')

In [28]:
# test local
init_context(context)
event = ""#nuclio.Event(body='')
df = handler(context,event)

Python> 2021-06-29 08:30:05,086 [info] Initalizing context & constructing featrure vector
Python> 2021-06-29 08:30:05,394 [info] Reading features from feature vector


FileNotFoundError: '/projects/stocks2-dani/FeatureStore/news/parquet/sets/news-latest' not found

In [19]:
df

[{'columns': [{'text': 'Opens_min_1h', 'type': 'object'},
   {'text': 'Opens_max_1h', 'type': 'object'},
   {'text': 'Volumes_min_1h', 'type': 'object'},
   {'text': 'Volumes_max_1h', 'type': 'object'},
   {'text': 'Open', 'type': 'object'},
   {'text': 'High', 'type': 'object'},
   {'text': 'Low', 'type': 'object'},
   {'text': 'Close', 'type': 'object'},
   {'text': 'Volume', 'type': 'object'},
   {'text': 'Sentiment', 'type': 'object'}],
  'rows': array([[ 2.66500000e+02,  2.66500000e+02,  7.12949000e+05, ...,
           2.66749908e+02,  7.12949000e+05,  6.66666667e-01],
         [ 2.66500000e+02,  2.66500000e+02,  7.12949000e+05, ...,
           2.66749908e+02,  7.12949000e+05,  6.66666667e-01],
         [ 2.66500000e+02,  2.66500000e+02,  7.12949000e+05, ...,
           2.66749908e+02,  7.12949000e+05,  6.66666667e-01],
         ...,
         [ 2.67760010e+02,  2.68869995e+02,  2.04360000e+04, ...,
           2.68720001e+02,  3.86417000e+05, -1.66666667e+00],
         [ 2.43416992

In [9]:
# test remote deployment
from mlrun import code_to_function
fn = code_to_function('vector_reader',
                      handler='handler')

# Set parameters for current deployment
fn.set_envs({'PROJECT_NAME' : "stocks-" + os.getenv('V3IO_USERNAME'),
             'STOCKS_STREAM':  os.getenv('V3IO_USERNAME') +' /stocks/stocks_stream'})
fn.spec.max_replicas = 2

In [10]:
addr = fn.deploy(project="stocks-" + os.getenv('V3IO_USERNAME'))

> 2021-06-28 07:26:08,310 [info] Starting remote function deploy
2021-06-28 07:26:08  (info) Deploying function
2021-06-28 07:26:08  (info) Building
2021-06-28 07:26:08  (info) Staging files and preparing base images
2021-06-28 07:26:08  (info) Building processor image
2021-06-28 07:26:22  (info) Build complete
2021-06-28 07:26:30  (info) Function deploy complete
> 2021-06-28 07:26:31,698 [info] function deployed, address=default-tenant.app.vmdev31.lab.iguazeng.com:30993


In [ ]:
!curl {addr}